# Pokušaj poboljšanja bota
Tomislav Žiger

In [1]:
import requests
import models
import os
from dotenv import load_dotenv
import util
import bot_my_version
import pandas as pd

In [2]:
data_handler = util.DataHandler()
data_augmenter = util.DataAugmenter()

Dodajem pomoćnu metodu za accuracy, ovdje ću lokalno testirati bota jer mi je jednostavnije, ali naravno možemo zamijeniti originalnog bota i hostati app

In [3]:
def get_accuracy(x, y, bot_version):
    count = 0
    for message, intent in zip(x, y):
        pred_response = bot_version.predict(message)
        pred = models.PromptResponse(**pred_response)
        if pred.intent == intent:
            count += 1
    return count / len(x)

Ovdje spajamo train dataset i "teški" test dataset (dataset_test_hard.csv), pa želimo vidjeti hoće li model bolje radit od  77.27 % koliko je imao kad je samo bil treniran na train datasetu i onda testiran na "dataset_test_easy.csv"


In [4]:
x1, y1 = data_handler.get_train_data_xy()
x2, y2 = data_handler.get_test_data_hard_xy()

x = pd.concat([x1, x2], ignore_index=True)
y = pd.concat([y1, y2], ignore_index=True)
bot_my_version._bundle = bot_my_version._build_pipeline(X = x,y = y)


making new pipeline, x: 85 y: 85


/Users/tomislavziger/Developer/rba-chatbot-assignment/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [5]:
x,y = data_handler.get_test_data_easy_xy()
print("Točnost: ", get_accuracy(x,y, bot_my_version) * 100 , "%")


Točnost:  86.36363636363636 %


Evo sa 77.27% na 86.36% smo uspjeli točnost povećati samo sa većim datasetom


Zanimljivo bi bilo testirati kakva je točnost na test_hard ako treniramo sa test_easy i train, samo sa train je bila jako loša točnost 22.22%


In [6]:
x1, y1 = data_handler.get_train_data_xy()
x2, y2 = data_handler.get_test_data_easy_xy()

x = pd.concat([x1, x2], ignore_index=True)
y = pd.concat([y1, y2], ignore_index=True)
bot_my_version._bundle = bot_my_version._build_pipeline(X = x,y = y)

x,y = data_handler.get_test_data_hard_xy()
print("Točnost :", get_accuracy(x,y,bot_my_version) * 100 , "%")


making new pipeline, x: 44 y: 44
Točnost : 41.269841269841265 %


/Users/tomislavziger/Developer/rba-chatbot-assignment/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


vidimo da i ovdje veći dataset daje bolje rezultate od 41.27 % u odnosu na prijašnjih 22.22% kad je trenirano samo s originalnim train datasetom



Sad kad smo vidli da veći dataset daje bolje rezultate, idemo spojiti sve u jedan veliki dataset i nasumicno podijelit na train i test, prije nego krenemo dalje

In [7]:
from sklearn.model_selection import train_test_split

x, y = data_handler.get_combined_data_xy()

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=42, stratify=y)

print("duljina train skupa", len(X_train))
print("duljina test skupa",len(X_test))

duljina train skupa 85
duljina test skupa 22


i sad vidimo koji je rezultat za nasumičnu podjelu

In [8]:
bot_my_version._bundle = bot_my_version._build_pipeline(X = X_train,y = y_train)
print("Accuracy :", get_accuracy(X_test,y_test, bot_my_version) * 100 , "%")

making new pipeline, x: 85 y: 85
Accuracy : 72.72727272727273 %


/Users/tomislavziger/Developer/rba-chatbot-assignment/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


### Proširivanje skupa podataka

Sada idemo krenuti proširivati naš skup podataka u nadi da će poboljšati naše rezultate

In [9]:
data_augmenter = util.DataAugmenter()

print(type(X_train))

<class 'pandas.core.series.Series'>


Ovdje ćemo probati jednostavno proširenje dodati ćemo u train da maknemo samo prvo slovo sa metodom remove_first() i dodamo u train set

In [10]:
x, y = data_handler.get_combined_data_xy()

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=42, stratify=y)

bot_my_version._bundle = bot_my_version._build_pipeline(X = X_train, y = y_train)
print("Točnost prije augmentacije train :", get_accuracy(X_test,y_test, bot_my_version) * 100 , "%")

#augment
x,y = data_augmenter.remove_first(X_train,y_train)

X_train = pd.concat([X_train, x], ignore_index=True)
y_train = pd.concat([y_train, y], ignore_index=True)

bot_my_version._bundle = bot_my_version._build_pipeline(X = X_train, y = y_train)

print("Točnost poslije augmentacije train :", get_accuracy(X_test,y_test,bot_my_version) * 100 , "%")

    

making new pipeline, x: 85 y: 85
Točnost prije augmentacije train : 72.72727272727273 %
making new pipeline, x: 170 y: 170
Točnost poslije augmentacije train : 68.18181818181817 %


/Users/tomislavziger/Developer/rba-chatbot-assignment/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/tomislavziger/Developer/rba-chatbot-assignment/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Vidimo da je to pogoršalo naš rezultat, pa ajmo probati neki drugi pristup

Sljedeće za svaku riječ izbacujemo slovo jedno, da simuliramo kako bi korisnik mogao pogriješiti u pisanju rečenice

Primjer:
<br> 
prije: "Kolika je cijena ulaznice?"
<br>
nakon: "olika je cijena ulaznice?", "Klika je cijena ulaznice?"....



In [11]:
x, y = data_handler.get_combined_data_xy()

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=42, stratify=y)

bot_my_version._bundle = bot_my_version._build_pipeline(X = X_train, y = y_train)
print("Točnost prije augmentacije train :", get_accuracy(X_test,y_test,bot_my_version) * 100 , "%")

#augment
x,y = data_augmenter.remove_one(X_train,y_train)

X_train = pd.concat([X_train, x], ignore_index=True)
y_train = pd.concat([y_train, y], ignore_index=True)

bot_my_version._bundle = bot_my_version._build_pipeline(X = X_train, y = y_train)

print("Točnost poslije augmentacije train :", get_accuracy(X_test,y_test,bot_my_version) * 100 , "%")

    

making new pipeline, x: 85 y: 85
Točnost prije augmentacije train : 72.72727272727273 %
making new pipeline, x: 2695 y: 2695


/Users/tomislavziger/Developer/rba-chatbot-assignment/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/tomislavziger/Developer/rba-chatbot-assignment/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Točnost poslije augmentacije train : 68.18181818181817 %


Izgleda da izbacivanje slova iz rečenica ne poboljšava rezultat

Sljedeće što ću isprobati je promjena hrvatskih znakova(č,ć,ž,š,đ) mnogi korisnici ih ne pišu ili imaju tipkovnice koje nemaju te znakove.

Primjer: 
<br>
prije: "Jel ima u muzeju kafić?"
<br>
nakon: "Jel ima u muzeju kafic?"




In [12]:
x, y = data_handler.get_combined_data_xy()

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=42, stratify=y)

bot_my_version._bundle = bot_my_version._build_pipeline(X = X_train, y = y_train)
print("Točnost prije augmentacije train :", get_accuracy(X_test,y_test,bot_my_version) * 100 , "%")

#augment
x,y = data_augmenter.remove_croatian(X_train,y_train)

X_train = pd.concat([X_train, x], ignore_index=True)
y_train = pd.concat([y_train, y], ignore_index=True)

bot_my_version._bundle = bot_my_version._build_pipeline(X = X_train, y = y_train)

print("Točnost poslije augmentacije train :", get_accuracy(X_test,y_test,bot_my_version) * 100 , "%")

making new pipeline, x: 85 y: 85
Točnost prije augmentacije train : 72.72727272727273 %
making new pipeline, x: 120 y: 120
Točnost poslije augmentacije train : 54.54545454545454 %


/Users/tomislavziger/Developer/rba-chatbot-assignment/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/tomislavziger/Developer/rba-chatbot-assignment/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


S obzirom da se proširivanje skupa podataka za sadašnji model nije pokazalo kao dobro rješenje ,idemo sad probat drukčiji model od multinomijalne regresije

Koristit ću XGboost, jedan od popularnijih modela za klasifikaciju, kopirao sam kod bot.py u bot_new_model.py i promijenio regresiju u xgboost



In [13]:
import bot_new_model

x, y = data_handler.get_combined_data_xy()

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=42, stratify=y)

bot_new_model._bundle = bot_new_model._build_pipeline(X = X_train, y = y_train, classifier="xgb")
print("Točnost prije augmentacije train :", get_accuracy(X_test,y_test,bot_new_model) * 100 , "%")

making new pipeline, x: 85 y: 85
Točnost prije augmentacije train : 22.727272727272727 %


In [14]:
x,y = data_augmenter.remove_one(X_train,y_train)

X_train = pd.concat([X_train, x], ignore_index=True)
y_train = pd.concat([y_train, y], ignore_index=True)

bot_new_model._bundle = bot_new_model._build_pipeline(X = X_train, y = y_train,classifier="xgb")

print("Točnost poslije augmentacije train :", get_accuracy(X_test,y_test,bot_new_model) * 100 , "%")

    

making new pipeline, x: 2695 y: 2695
Točnost poslije augmentacije train : 63.63636363636363 %


Vidimo da ne dobivamo bolje rezultate od regresije

Pa idemo probati sljedeće svm:

In [15]:
x, y = data_handler.get_combined_data_xy()

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=42, stratify=y)

bot_new_model._bundle = bot_new_model._build_pipeline(X = X_train, y = y_train, classifier = "svm")
print("Točnost prije augmentacije train :", get_accuracy(X_test,y_test,bot_new_model) * 100 , "%")

making new pipeline, x: 85 y: 85
Točnost prije augmentacije train : 63.63636363636363 %


In [16]:
x,y = data_augmenter.remove_one(X_train,y_train)

X_train = pd.concat([X_train, x], ignore_index=True)
y_train = pd.concat([y_train, y], ignore_index=True)

bot_new_model._bundle = bot_new_model._build_pipeline(X = X_train, y = y_train, classifier = "svm")
print("Točnost poslije augmentacije train :", get_accuracy(X_test,y_test,bot_new_model) * 100 , "%")

making new pipeline, x: 2695 y: 2695
Točnost poslije augmentacije train : 63.63636363636363 %


Još zadnje što ćemo probat je RandomForestClassifier

In [19]:
x, y = data_handler.get_combined_data_xy()

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=42, stratify=y)

bot_new_model._bundle = bot_new_model._build_pipeline(X = X_train, y = y_train, classifier = "rf")
print("Točnost prije augmentacije train :", get_accuracy(X_test,y_test,bot_new_model) * 100 , "%")

making new pipeline, x: 85 y: 85
Točnost prije augmentacije train : 50.0 %


In [20]:
x,y = data_augmenter.remove_one(X_train,y_train)

X_train = pd.concat([X_train, x], ignore_index=True)
y_train = pd.concat([y_train, y], ignore_index=True)

bot_new_model._bundle = bot_new_model._build_pipeline(X = X_train, y = y_train, classifier = "rf")
print("Točnost poslije augmentacije train :", get_accuracy(X_test,y_test,bot_new_model) * 100 , "%")

making new pipeline, x: 2695 y: 2695
Točnost poslije augmentacije train : 27.27272727272727 %
